**`This code shows how to use FAISS and LangChain to build a generative QA model or chatbot, using the model: gpt-4-1106-preview. `**

Dataset: CGSQuAD

Dataset format: XLSX

Two main components are required: **prompt** and **response**. The richer the response, the better the generated answer is. The **"context"** is used as the input column, as the model performs similarity search (using embeddings indexing)

* Dataset file name: CGSQuAD.xlsx
* Necessary columns: context
* Optional columns: question, answers
* Unecessary columns: answer_start, pq_id
* Output file name: similarity_evaluation_GPT_4_Turbo.csv, similarity_evaluation_GPT_4_Turbo.xlsx

#Install dependencies

In [1]:
!pip install langchain
!pip install openai
!pip install colab-env -qU
!pip install tiktoken
!pip install faiss-gpu
!pip install -U sentence-transformers
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver 

#Use an OpenAI API key

Requires an OpenAI account: https://platform.openai.com/



In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

#Test  some questions to make sure the API works

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")

In [ ]:
question = "ماهي أصعب لغة برمجة في العالم؟"
print(llm(question))



اللغة البرمجية الأصعب في العالم هي لغة الآلة C.


In [ ]:
question = "What are the top 4 resources to learn Golang in 2023?"
print(llm(question))



1. Go by Example: This is a great resource for learning Go, as it provides example code for each concept, making it easy to understand and apply to real world use cases.

2. A Tour of Go: This is an interactive tour of the Go programming language by the official Go team. It introduces the language and its concepts in a step-by-step manner.

3. Golang Tutorials: This website provides a number of tutorials and examples to help you learn Go. It also provides some tips and tricks to help you write better Go code.

4. GoDoc: This is an official documentation site for Go, where you can find detailed information about the language, its syntax, and its libraries.


#Create an auto prompt and chain it to the model

In [ ]:
from langchain import PromptTemplate
template = "What are the top {k} resources to learn {this} in 2023?"
prompt = PromptTemplate(template=template,input_variables=['k','this'])

from langchain import LLMChain
chain = LLMChain(llm=llm,prompt=prompt)

In [ ]:
input = {'k':3,'this':'cooking'}
print(chain.run(input))



1. YouTube – With its vast library of instructional cooking videos, YouTube is an excellent resource for learning how to cook.

2. Web-based cooking classes – Online cooking classes are becoming increasingly popular, as they provide an interactive way to learn about cooking techniques and recipes.

3. Cookbooks – Cookbooks are a great way to learn cooking basics as well as more advanced techniques. Look for cookbooks that are tailored to your experience level and interests.


#Import Facebook FAISS and LangChain packages

In [3]:
import os, dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain import PromptTemplate

#Use minimized unique contexts to reduce costs

In [4]:
import pandas as pd
import re
import heapq
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
data_file = "/content/CGSQuAD.xlsx"
data = pd.read_excel(data_file)
def extract_relevant_sentences(text, answer, answer_start, n_sentences=2):
    sentences = text.split('. ')#'. ' as a sentence slicer

    answer_start = answer_start#get answer location
    answer_end = answer_start + len(str(answer))

    start_index = 0#identify start and end of relevant context
    end_index = len(sentences)

    for i, sentence in enumerate(sentences):
        if sentence.lower().find(str(answer).lower()) != -1:
            counter = 1
            while i - counter >= 0 and counter <= n_sentences:#get n sentences from right and left
                start_index = i - counter
                counter += 1

            counter = 1
            while i + counter < len(sentences) and counter <= n_sentences:
                end_index = i + counter + 1
                counter += 1
            break

    new_context = '. '.join(sentences[start_index:end_index])#construct the relevant context out of the extracted sentences

    if not new_context.endswith('.'):#add a period at the last sentence
        new_context += '.'

    return new_context

data['context'] = data.apply(lambda row: extract_relevant_sentences(row['context'], row['answers'], row['answer_start']), axis=1)#for each context, call the function
data.to_csv("minimized_contexts.csv", index=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
data['context'][0]

'رسالة كلية الدراسات العليا هي العمل المخطط الهادف الى المساهمة في تنمية إمكانات جامعة الكويت لتكون مؤسسة علمية تعليمية متميزة. أهداف كلية الدراسات العليا تشمل اتاحة فرص تعليم ومواكبة التطور العالمي وصنع المثقف العصري وتناول قضايا المجتمع الكويتي. مهام كلية الدراسات العليا تشمل الموافقة على برامج الدراسات العليا ووضع الأنظمة وتحديد أسس القبول وعمل التقويم الدوري وطرح برامج اختصاصية.'

In [6]:
import pandas as pd

df=data
unique_contexts = df["context"].unique()#extract unique contexts
unique_df = pd.DataFrame({"context": unique_contexts})#new df to hold the unique contexts
print(unique_df.count())
output_file = "unique_contexts.csv"
unique_df.to_csv(output_file, index=False)

context    272
dtype: int64


#Calculate the number of tokens using NLTK and OpenAI Tiktoken

This is a method that helps in estimating billings. Detailed description of the billings of the made API requests are found here: https://platform.openai.com/account/usage

In [7]:
import pandas as pd
import nltk
import tiktoken
nltk.download('perluniprops')
nltk.download('nonbreaking_prefixes')
nltk.download('punkt')

from nltk.tokenize import word_tokenize

num_tokens_list_nltk = []
num_tokens_list_tiktoken = []
i=1
sum=0
for context in df["context"].unique():
    print("Context: ", i)
    tokens_nltk = word_tokenize(context)#tokenize
    num_tokens_nltk = len(tokens_nltk)
    print("NLTK: ", num_tokens_nltk)
    num_tokens_list_nltk.append(num_tokens_nltk)

    #choose model first, each OpenAI model has different toenization method
    #gpt-4 and gpt-3.5-turbo-16k have the same tokenizer
    tiktoken_encoding = tiktoken.encoding_for_model("gpt-4")
    tokens_tiktoken = tiktoken_encoding.encode(context)#tokenize
    num_tokens_tiktoken = len(tokens_tiktoken)
    sum = sum + num_tokens_tiktoken
    print("Tiketoken: ", num_tokens_tiktoken)
    num_tokens_list_tiktoken.append(num_tokens_tiktoken)
    i=i+1
print("Total Tiktokens: ", sum)

[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Context:  1
NLTK:  62
Tiketoken:  262
Context:  2
NLTK:  78
Tiketoken:  331
Context:  3
NLTK:  97
Tiketoken:  416
Context:  4
NLTK:  95
Tiketoken:  412
Context:  5
NLTK:  96
Tiketoken:  417
Context:  6
NLTK:  95
Tiketoken:  407
Context:  7
NLTK:  94
Tiketoken:  395
Context:  8
NLTK:  92
Tiketoken:  370
Context:  9
NLTK:  83
Tiketoken:  319
Context:  10
NLTK:  73
Tiketoken:  261
Context:  11
NLTK:  70
Tiketoken:  250
Context:  12
NLTK:  73
Tiketoken:  259
Context:  13
NLTK:  74
Tiketoken:  270
Context:  14
NLTK:  74
Tiketoken:  275
Context:  15
NLTK:  74
Tiketoken:  278
Context:  16
NLTK:  71
Tiketoken:  270
Context:  17
NLTK:  72
Tiketoken:  284
Context:  18
NLTK:  68
Tiketoken:  265
Context:  19
NLTK:  83
Tiketoken:  341
Context:  20
NLTK:  86
Tiketoken:  345
Context:  21
NLTK:  83
Tiketoken:  344
Context:  22
NLTK:  89
Tiketoken:  379
Context:  23
NLTK:  98
Tiketoken:  425
Context:  24
NLTK:  96
Tiketoken:  420
Context:  25
NLTK:  98
Tiketoken:  436
Context:  26
NLTK:  97
Tiketoken: 

#Load dataset and index it using FAISS

In [8]:
loader = CSVLoader("/content/unique_contexts.csv")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

faissIndex = FAISS.from_documents(docs, OpenAIEmbeddings())
faissIndex.save_local("graduate_studies_docs")#saves a folder to hold index.faiss and index.pkl

#Create the QA chatbot by injecting CGSQuAD to the model

Maximum number of tokens for any generated output = 75

In [9]:
dotenv.load_dotenv()
chatbot = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        temperature=0, model_name="gpt-4-1106-preview", max_tokens=75
    ),
    chain_type="stuff",
    retriever=FAISS.load_local("graduate_studies_docs", OpenAIEmbeddings())
        .as_retriever(search_type="similarity", search_kwargs={"k":1})
)
template = """
respond as succinctly as possible. {query}?
"""
prompt = PromptTemplate(
    input_variables=["query"],
    template=template,
)

#Test and evaluate the model on CGSQuAD

Asking the same question exactly. Same generated answers indicate a stable and strong model.

In [10]:
print(chatbot.run(
    prompt.format(query=" ما هي شروط القبول؟"))
)

شروط القبول تتضمن استيفاء المعدل المطلوب للقبول النظامي، وفي حالة القبول المشروط يجب على الطالب اجتياز مواد دراسية محدد


In [11]:
print(chatbot.run(
    prompt.format(query=" ما هي شروط القبول؟"))
)

شروط القبول تتضمن استيفاء المعدل المطلوب للقبول النظامي، وفي حالة القبول المشروط يجب على الطالب اجتياز مواد دراسية محدد


Asking an unpopular/broad question

In [12]:
print(chatbot.run(
    prompt.format(query=" ما هي شروط القبول  إذا كان المتقدم من فئة المعاقين ؟"))
)

معدل عام 2.33.


Asking the same question differently

In [13]:
print(chatbot.run(
    prompt.format(query=" كم هو معدل القبول للطالب المقيد؟"))
)

معدل القبول للطالب المقيد هو 2.67 نقاط من أصل 4.00 نقاط.


In [14]:
print(chatbot.run(
    prompt.format(query=" ما هو معدل القبول المطلوب للطالب المقيد؟"))
)

معدل القبول المطلوب للطالب المقيد هو 2.67 نقاط من أصل 4.00 نقاط.


*In here, the same question is asked, however, the model gave a different answer. The answer is not wrong, the model did not understand the question as it it not clear. The question needs to specify: is it the GPA of applying to the CGS or the required GPA to change student status from unenrolled into enrolled student. These are two different cases and the model thought this refers to the second case. The answer is correct if that is the intended question*


In [15]:
print(chatbot.run(
    prompt.format(query=" ما هو المعدل المطلوب للطالب المقيد؟"))
)

المعدل المطلوب هو 2.67 نقطة.


Asking a question that has no explicit answer within the context

In [16]:
print(chatbot.run(
    prompt.format(query=" ما الفرق بين الطالب المقيد والغير مقيد؟"))
)

الطالب المقيد استوفى جميع الشروط بما فيها المعدل المطلوب للقبول، بينما الطالب الغير مقيد لم يستوفِ شرط المعدل ويقب


Asking questions that might appear similar but are not

In [17]:
print(chatbot.run(
    prompt.format(query=" ما هو الطالب المقيد؟"))
)

الطالب المقيد هو الطالب المستوفي لجميع الشروط.


In [18]:
print(chatbot.run(
    prompt.format(query=" الطالب الغير مقيد هو؟"))
)

الطالب الغير مقيد هو الطالب الغير مستوفي لشرط المعدل المطلوب للقبول ويتم قبوله قبولا مشروطا.


In [19]:
print(chatbot.run(
    prompt.format(query=" ما هو الطالب غير المقيد؟"))
)

الطالب غير المقيد هو الطالب الذي لم يستوفِ شرط المعدل المطلوب للقبول ويتم قبوله مشروطاً حتى يجتاز 12 وحدة دراس


In [20]:
print(chatbot.run(
    prompt.format(query=" ما هو معدل القبول المطلوب من الطالب غير المقيد؟"))
)

معدل القبول المطلوب من الطالب غير المقيد هو 2.50 نقاط من أصل 4.00 نقاط.


In [21]:
print(chatbot.run(
    prompt.format(query="what are the offered programs? "))
)

The offered programs can be found on the website of the College of Graduate Studies at Kuwait University: http://www.kuniv.edu.kw/COGS/index.htm.


In [22]:
print(chatbot.run(
    prompt.format(query="where to apply for IELTS?"))
)

يمكن التقديم لاختبار IELTS من خلال معهد IDP.


In [23]:
print(chatbot.run(
    prompt.format(query="how much is the allowance for the external examiner?"))
)

The allowance for the external examiner is 500 Kuwaiti Dinars.


In [24]:
print(chatbot.run(
    prompt.format(query="what is an enrolled student?"))
)

الطالب المقيد هو الطالب المستوفي لجميع الشروط.


In [25]:
print(chatbot.run(
    prompt.format(query="what is an unenrolled student?"))
)

الطالب غير المقيد هو الطالب الذي لم يستوفِ شرط المعدل المطلوب.


In [12]:
print(chatbot.run(
    prompt.format(query="what is the GPA required to apply as an enrolled student? answer in english"))
)

The GPA required to apply as an enrolled student is 2.67 out of 4.00.


Test the model on n questions. Use manual evaluation & semantic similarity check. Sometimes the API call disconnects

In [26]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import torch

def calculate_similarity(sentence1, sentence2):
    # Ensure sentence1 and sentence2 are treated as strings
    sentence1 = str(sentence1)
    sentence2 = str(sentence2)

    # Calculate embeddings
    model = SentenceTransformer("symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli")
    embedding1 = model.encode(sentence1, convert_to_tensor=True)
    embedding2 = model.encode(sentence2, convert_to_tensor=True)

    # Calculate cosine similarity between embeddings
    similarity_score = torch.nn.functional.cosine_similarity(embedding1.unsqueeze(0), embedding2.unsqueeze(0)).item()
    return similarity_score

# Your dataframe and other code

dfx = df  # Assuming df is defined somewhere

questions = dfx["question"]
answers = dfx["answers"]

random_questions = questions.sample(n=1500, random_state=42)

random_question = []
generated_responses = []
corresponding_responses = []
similarity_scores = []

pbar = tqdm(total=len(random_questions), desc="Calculating Similarity")

for question in random_questions:
    generated_response = chatbot.run(prompt.format(query=question))
    random_question.append(question)
    generated_responses.append(generated_response)

    # Use boolean indexing to find the corresponding response
    corresponding_response = answers[questions == question].values[0]
    corresponding_responses.append(corresponding_response)

    similarity = calculate_similarity(generated_response, corresponding_response)
    similarity_scores.append(similarity)

    pbar.update(1)
pbar.close()

mean_similarity_score = np.mean(similarity_scores)

print("Mean Similarity Score:", mean_similarity_score)

evaluation_df = pd.DataFrame({
    "random_question": random_question,
    "generated_response": generated_responses,
    "corresponding_response": corresponding_responses,
    "similarity_score": similarity_scores
})


Calculating Similarity:   0%|          | 0/1500 [00:00<?, ?it/s]

Calculating Similarity: 100%|██████████| 1500/1500 [2:33:38<00:00,  6.15s/it]

Mean Similarity Score: 0.5230373931688567


In [28]:
evaluation_df.to_csv("similarity_evaluation_GPT_4_Turbo.csv", index=False)

In [29]:
evaluation_df.to_excel("similarity_evaluation_GPT_4_Turbo.xlsx", index=False)

In [27]:
evaluation_df

,random_question,generated_response,corresponding_response,similarity_score
0,كيف تتألف لجنة البرنامج؟,تتألف لجنة البرنامج من ثلاثة أعضاء على الأقل، ...,عن طريق عميد كلية الدراسات العليا,0.350557
1,كم عدد المشاركات العلمية التي يستطيع الطالب حض...,مهمة واحدة فقط.,مهمة واحدة فقط خلال مدة دراسته,0.489495
2,هل يشترط اعداد الرسالة للحصول على دبلوم دراسات...,لا أعلم.,ولا يشترط اجتياز الامتحان الشامل او اعداد أطروحة,-0.063879
3,ما هي قيمة بدل السفر اليومي لتكاليف الرحلات ال...,بدل السفر اليومي هو 30 دينار كويتي.,30 دينار كويتي,0.785821
4,ماذا يحصل اذا سقط الطالب في اختبار اطروحة الدك...,يمنح الطالب فرصة أخرى.,يمنح فرصة أخرى,0.680105
...,...,...,...,...
1495,هل يشترط حضور مختبر اختبار بحث الدكتوراة اثناء...,نعم يشترط حضوره.,نعم يشترط حضور ممتحن امتحان رسالة الدكتوراة,0.326360
1496,ما هي شروط القبول المشروط للماجستير إذا كان ال...,معدل عام 2.33.,معدل عام 2.33,0.979141
1497,كم قيمة مكافأة لجنة الاختبار الشفهي لكلية الحقوق؟,قيمة مكافأة لجنة الاختبار الشفهي لكلية الحقوق ...,100 دينار كويتي لكل عضو,0.889421
1498,ما هو المعدل الادنى للتخرج من دبلوم الدراسات ا...,المعدل الأدنى المطلوب للتخرج من الدبلوم العالي...,هو 2.67,0.548208
